In [3]:
import numpy as np

data = np.load('./res/tmp/plenoptic/coffee_martini/poses_bounds.npy')
data.shape

(18, 17)

In [4]:
def get_poses(data):
    mat, close, far = data[:, :-2].reshape((-1, 3, 5)), data[:, -2], data[:, -1]
    T, mat2 = mat[:, :, :-1], mat[:, :, -1:]
    h, w, f = mat2[:, 0, 0], mat2[:, 1, 0], mat2[:, 2, 0]

    return T, (h, w), (close, far), f

T, (h, w), (c, f), F = get_poses(data)

T.shape, (h.shape, w.shape), (c.shape, f.shape), F.shape


((18, 3, 4), ((18,), (18,)), ((18,), (18,)), (18,))

In [5]:
data[0, :], T[0, :, :], (h[0], w[0]), (c[0], f[0]), F[0]


(array([-2.17321701e-02,  9.94349165e-01, -1.03910786e-01,  4.43964684e-01,
         2.02800000e+03,  9.99753096e-01,  2.11326529e-02, -6.86712777e-03,
        -1.10350344e+00,  2.70400000e+03, -4.63241219e-03, -1.04034368e-01,
        -9.94562915e-01, -3.49927298e-01,  1.46075434e+03,  8.83138423e+00,
         1.09775424e+02]),
 array([[-0.02173217,  0.99434916, -0.10391079,  0.44396468],
        [ 0.9997531 ,  0.02113265, -0.00686713, -1.10350344],
        [-0.00463241, -0.10403437, -0.99456291, -0.3499273 ]]),
 (2028.0, 2704.0),
 (8.831384232013642, 109.77542390000633),
 1460.7543404163334)

In [ ]:
def load_data(basedir, factor=None, width=None, height=None):
    # Taken from repos/LLFF/llff/poses/pose_utils.py
    
    poses_arr = np.load(os.path.join(basedir, 'poses_bounds.npy'))
    poses = poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1,2,0])
    bds = poses_arr[:, -2:].transpose([1,0])
    
    img0 = [os.path.join(basedir, 'images', f) for f in sorted(os.listdir(os.path.join(basedir, 'images'))) \
            if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')][0]
    sh = imageio.imread(img0).shape
    
    sfx = ''
    
    # just resizes the images and sends to a new folder with suffix images_{}x{} or images_{}
    if factor is not None:
        sfx = '_{}'.format(factor)
        minify(basedir, factors=[factor])
        factor = factor
    elif height is not None:
        factor = sh[0] / float(height)
        width = int(sh[1] / factor)
        minify(basedir, resolutions=[[height, width]])
        sfx = '_{}x{}'.format(width, height)
    elif width is not None:
        factor = sh[1] / float(width)
        height = int(sh[0] / factor)
        minify(basedir, resolutions=[[height, width]])
        sfx = '_{}x{}'.format(width, height)
    else:
        factor = 1
    
    imgdir = os.path.join(basedir, 'images' + sfx)
    if not os.path.exists(imgdir):
        print( imgdir, 'does not exist, returning' )
        return
    
    imgfiles = [os.path.join(imgdir, f) for f in sorted(os.listdir(imgdir)) if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')]
    if poses.shape[-1] != len(imgfiles):
        print( 'Mismatch between imgs {} and poses {} !!!!'.format(len(imgfiles), poses.shape[-1]) )
        return
    
    # new data after resizing is width/factor, height/factor, focal/factor (factor 2 halves the image, 4 gets 1/4th of the size, and so on)
     sh = imageio.imread(imgfiles[0]).shape
    poses[:2, 4, :] = np.array(sh[:2]).reshape([2, 1])
    poses[2, 4, :] = poses[2, 4, :] * 1./factor
    
    return poses, bds
